In [1]:
import numpy as np
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
#Extract data from CSV
df1=pd.read_csv("earthquake_data.csv")
#col1 = df1[['Date','Time','Latitude','Longitude']]
col1 = df1[['Latitude','Longitude','Depth']]
col2 = df1['Magnitude']
#Convert to Numpy array
InputX1 = col1.as_matrix()
InputY1 = col2.as_matrix()
print(InputX1)
InputX1.astype(float, copy=False)
InputY1.astype(float, copy=False)


[[  19.246   145.616   131.6   ]
 [   1.863   127.352    80.    ]
 [ -20.579  -173.972    20.    ]
 ..., 
 [  36.9179  140.4262   10.    ]
 [  -9.0283  118.6639   79.    ]
 [  37.3973  141.4103   11.94  ]]


array([ 6. ,  5.8,  6.2, ...,  5.9,  6.3,  5.5])

In [3]:
#Min-max Normalization
X1_min = np.amin(InputX1,0)     
X1_max = np.amax(InputX1,0)   
print("Mininum values:",X1_min)
print("Maximum values:",X1_max)
Y1_min = np.amin(InputY1)     
Y1_max = np.amax(InputY1) 
InputX1_norm = (InputX1-X1_min)/(X1_max-X1_min)
InputY1_norm = InputY1  #No normalization in output
#InputY1_norm = (InputY1-Y1_min)/(Y1_max-Y1_min)

#Reshape
Xfeatures = 3 #Number of input features
Yfeatures = 1 #Number of input features
samples = 23000 # Number of samples

InputX1_reshape = np.resize(InputX1_norm,(samples,Xfeatures))
InputY1_reshape = np.resize(InputY1_norm,(samples,Yfeatures))
#print("X1 normalized:",InputX1_reshape)
#print("Y1 normalized:",InputY1_reshape)

Mininum values: [ -77.08  -179.997   -1.1  ]
Maximum values: [  86.005  179.998  700.   ]


In [4]:
#Training data
batch_size = 20000
InputX1train = InputX1_reshape[0:batch_size,:]
InputY1train = InputY1_reshape[0:batch_size,:]
#Validation data
v_size = 2500
InputX1v = InputX1_reshape[batch_size:batch_size+v_size,:]
InputY1v = InputY1_reshape[batch_size:batch_size+v_size,:]
#print(InputX1v)

In [5]:
#Network hyper parametres
learning_rate = 0.001
training_iterations = 100000
display_iterations = 20000

In [6]:
#Input
X = tf.placeholder(tf.float32,shape=(None,Xfeatures))#[batch size, input_features]
#Output
Y = tf.placeholder(tf.float32)

In [7]:
#Neurons
L1 = 3 #Number of neurons in 1st layer
L2 = 3 #Number of neurons in 2nd layer
L3 = 3 #Number of neurons in 2nd layer
#Layer1 weights
W_fc1 = tf.Variable(tf.random_uniform([Xfeatures,L1])) # [input_features,Number of neurons]) 
b_fc1 = tf.Variable(tf.constant(0.1,shape=[L1]))
#Layer2 weights
W_fc2 = tf.Variable(tf.random_uniform([L1,L2])) # [Number of neurons in preceding layer,Number of neurons]) 
b_fc2 = tf.Variable(tf.constant(0.1,shape=[L2]))
#Layer3 weights
W_fc3 = tf.Variable(tf.random_uniform([L2,L3])) # [Number of neurons in preceding layer,Number of neurons]) 
b_fc3 = tf.Variable(tf.constant(0.1,shape=[L3]))
#Output layer weights
W_fO= tf.Variable(tf.random_uniform([L3,Yfeatures])) #  [Number of neurons in preceding layer,output_features]) 
b_fO = tf.Variable(tf.constant(0.1,shape=[Yfeatures]))

In [8]:
#Layer 1
matmul_fc1=tf.matmul(X, W_fc1) + b_fc1
h_fc1 = tf.nn.relu(matmul_fc1)   #ReLU activation
#Layer 2
matmul_fc2=tf.matmul(h_fc1, W_fc2) + b_fc2
h_fc2 = tf.nn.relu(matmul_fc2)   #ReLU activation
#Layer 3
matmul_fc3=tf.matmul(h_fc2, W_fc3) + b_fc3
h_fc3 = tf.nn.relu(matmul_fc3)   #ReLU activation
#Output layer
matmul_fc4=tf.matmul(h_fc3, W_fO) + b_fO
output_layer = matmul_fc4  #linear activation

In [9]:
#Loss function
mean_square =  tf.reduce_mean(tf.square(Y-output_layer))
train_step = tf.train.AdamOptimizer(learning_rate).minimize(mean_square)
#train_step = tf.train.AdagradOptimizer(learning_rate).minimize(mean_square)
#train_step = tf.train.MomentumOptimizer(learning_rate,0.01).minimize(mean_square)
#train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(mean_square)
#Operation to save variables
saver = tf.train.Saver()

In [10]:
#Initialization and session
init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    print("Training loss:",sess.run([mean_square],feed_dict={X:InputX1train,Y:InputY1train}))
    for i in range(training_iterations):
        sess.run([train_step],feed_dict={X:InputX1train,Y:InputY1train})
        if i%display_iterations ==0:
            print("Training loss is:",sess.run([mean_square],feed_dict={X:InputX1train,Y:InputY1train}),"at itertion:",i)
            print("Validation loss is:",sess.run([mean_square],feed_dict={X:InputX1v,Y:InputY1v}),"at itertion:",i)
    # Save the variables to disk.
    save_path = saver.save(sess, "/tmp/earthquake_model.ckpt")
    print("Model saved in file: %s" % save_path)

    print("Final training loss:",sess.run([mean_square],feed_dict={X:InputX1train,Y:InputY1train}))
    print("Final validation loss:",sess.run([mean_square],feed_dict={X:InputX1v,Y:InputY1v}))
    

Training loss: [4.6381264]
Training loss is: [4.5309277] at itertion: 0
Validation loss is: [4.717792] at itertion: 0
Training loss is: [0.17617485] at itertion: 20000
Validation loss is: [0.18727431] at itertion: 20000
Training loss is: [0.17617068] at itertion: 40000
Validation loss is: [0.18727618] at itertion: 40000
Training loss is: [0.17616746] at itertion: 60000
Validation loss is: [0.18727437] at itertion: 60000
Training loss is: [0.17616335] at itertion: 80000
Validation loss is: [0.18727487] at itertion: 80000
Model saved in file: /tmp/earthquake_model.ckpt
Final training loss: [0.17616388]
Final validation loss: [0.18727615]


In [12]:
#Testing
lat = input("Enter Latitude between -77 to 86:")
long = input("Enter Longitude between -180 to 180:")
depth = input("Enter Depth between 0 to 700:")
InputX2 = np.asarray([[lat,long,depth]],dtype=np.float32)
InputX2_norm = (InputX2-X1_min)/(X1_max-X1_min)
InputX1test = np.resize(InputX2_norm,(1,Xfeatures))
with tf.Session() as sess:
    # Restore variables from disk for validation.
    saver.restore(sess, "/tmp/earthquake_model.ckpt")
    print("Model restored.")
    #print("Final validation loss:",sess.run([mean_square],feed_dict={X:InputX1v,Y:InputY1v}))
    print("output:",sess.run([output_layer],feed_dict={X:InputX1test}))

Enter Latitude between -77 to 86:45.71
Enter Longitude between -180 to 180:26.5
Enter Depth between 0 to 700:97
Model restored.
output: [array([[ 5.89945841]], dtype=float32)]
